# Data Science with Python and Neo4j

## Packages You'll Need

```
pip install py2neo
pip install ipython-cypher
pip install pandas
pip install networkx
pip install python-igraph
```

## Contents

### Basics

* [py2neo](py2neo.ipynb)
* [ipython-cypher](ipython-cypher.ipynb)
  - pandas
  - NetworkX
* [igraph](igraph.ipynb)